In [1]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
from bs4 import BeautifulSoup as soup
from pprint import pprint
from datetime import datetime, timedelta
import time as timer
import pandas as pd
import re

In [2]:
# Launch the browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:00<00:00, 24.3MB/s]


In [3]:
url = 'https://workspace.google.com/products/calendar/?hl=en-US'
browser.visit(url)
#Still need to log in manually, due to Google restrictions on scripts

In [4]:
url = 'https://calendar.google.com/calendar/u/0/r/search?q=ECD%20I'
browser.visit(url)

In [5]:
def calendar_scrape(search_url,course):
    session_dict = {'Subject':'','clock_hour':'','Start Time':'','End Time':'','Start Date':'','Location':''}
    subjects = []
    clock_hours = []
    start_times = []
    end_times = []
    start_dates = []
    locations = []

    #visits url and puts into a html_soup object
    browser.visit(search_url)
    timer.sleep(5)
    html = browser.html
    html_soup = soup(html, 'html.parser')

    div_elements = html_soup.find_all('div',{'role':'button','tabindex':'0'}, class_= 'i06k6b NlL62b')

    for element in div_elements:

        #Pull title from aria label
        title_dirty = element['aria-label']
        print(title_dirty)
        if "m, " in title_dirty:
            start_index = title_dirty.index("m, ")+ len("m, ")
        else:
            start_index = title_dirty.index("All day, ")+ len("All day, ")
        
        end_index = title_dirty.index(", Calendar:")

        title = title_dirty[start_index:end_index]
        subjects.append(title)
            
        if element:
            aria_label = element.get('aria-label')

            parts = aria_label.split(', ')
            month_date = parts[-2] if len(parts) > 1 else ""

            #separate month and date
            month, date = month_date.split(' ')
            if month == "January":
                month = 1
            elif month == "February":
                month= 2
            elif month == "March":
                month = 3
            elif month == "April":
                month= 4
            elif month == "May":
                month = 5
            elif month == "June":
                month= 6
            elif month == "July":
                month = 7
            elif month == "August":
                month= 8
            elif month == "September":
                month = 9
            elif month == "October":
                month = 10
            elif month == "November":
                month= 11
            elif month == "December":
                month = 12

            #pull year and put month, date, and year into whole_date variable. 
            year = parts[-1] if len(parts) > 1 else ""
            whole_date = f"{month}/{date}/{year}"

            #turn whole_date into datetime object
            date_object = datetime.strptime(whole_date,"%m/%d/%Y")


            

            #pulls start and end times
            time = aria_label.split(',')[0]
            if time == 'All day':
                time = ' 5pm to 6 pm'
            start_time, end_time = time.split(' to ')
            start_time_formatted = start_time.replace('pm',' PM').replace('am',' AM')
            end_time_formatted = end_time.replace('pm',' PM').replace('am',' AM')
            start_times.append(start_time_formatted)
            end_times.append(end_time_formatted)

            #temporal math
            start_time_math = start_time.replace('pm','').replace('am','')
            end_time_math = end_time.replace('pm','').replace('am','')
            #changes half hours to .5 and attaches to original
            start_time_math = start_time_math.replace(':30','.5')
            end_time_math = end_time_math.replace(':30','.5')
            start_time_math = start_time_math.replace(':45','.75')
            end_time_math = end_time_math.replace(':45','.75')
            start_time_math = start_time_math.replace(':15','.25')
            end_time_math = end_time_math.replace(':15','.25')
            #temporal math!
            clock_hrs = float(end_time_math) - float(start_time_math)
            clock_hrs

            #if the original clock hour formulation results in a negative number, it will
            #use anothe formulation to get proper clock hours, due to am/pm change
            if clock_hrs < 0:
                clock_hrs = (12-float(start_time_math)) + float(end_time_math)
            clock_hours.append(clock_hrs)

        
        start_dates.append(date_object)

        aria_label = element.get('aria-label')
        start_index = aria_label.find("Location: ")
        if start_index != -1:
            start_index += len("Location: ")
            end_index = aria_label.find(",", start_index)

            # Extract the location
            location = aria_label[start_index:end_index].strip()
            locations.append(location)
        else:
            location = "N/A"
            locations.append(location)




    
    session_dict['Start Date'] = start_dates
    session_dict['Start Time'] = start_times
    session_dict['End Time'] = end_times
    session_dict['clock_hour'] = clock_hours
    session_dict['Subject'] = subjects
    session_dict['Location'] = locations

    pprint(session_dict)
    session_df = pd.DataFrame(session_dict)
    session_df.to_csv(f'{course} Session List.csv')


In [8]:
calendar_scrape('https://calendar.google.com/calendar/u/0/r/search?q=Respiratory','Respiratory FA23')

8am to 10am, GA: Respiratory System Foundation - Mir, Calendar: DMD 2026 Curriculum Calendar, Accepted, Location: Check Description for Details, November 18, 2022
8am to 10am, Respiratory: Intro Lecture 1 & 2 - Overview - Borke, Calendar: DMD 2026 Curriculum Calendar, Accepted, Location: HEC Classroom D, November 29, 2023
10am to 12pm, Respiratory: Intro Lectures 3 & 4 - Overview - Borke, Calendar: DMD 2026 Curriculum Calendar, Accepted, Location: HEC Classroom D, November 29, 2023
1pm to 2pm, Respiratory: Special Presentation - Moghaddasijahromi, Calendar: DMD 2026 Curriculum Calendar, Accepted, Location: HEC Classroom D, November 30, 2023
2pm to 3pm, Respiratory: Learning Objectives Review 1A - Seleem, Calendar: DMD 2026 Curriculum Calendar, Accepted, Location: HEC Classroom D, November 30, 2023
3pm to 4pm, Respiratory: COPD/OSA - (Groups 1A, 2A, 3A, 4A), Calendar: DMD 2026 Curriculum Calendar, Accepted, Location: HEC 3rd Floor Study Rooms, November 30, 2023
3pm to 4pm, Respiratory: 